In [ ]:
%load_ext kedro.ipython

In [2]:
cd ../

/Users/Nok_Lam_Chan/dev/jaffle-shop


In [14]:
import ibis

con = catalog._datasets["raw_customers"].connection

In [5]:
customers = catalog.load("raw_customers")


[11/05/24 00:27:52] INFO     Loading data from raw_customers (TableDataset)...                  ]8;id=986419;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=323154;file:///Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 customers = catalog.load("raw_customers")                                                    │
│ ❱ 2 con = customers.connection                                                                   │
│   3                                                                                              │
│                                                                                                  │
│ /Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/ibis/expr/types/relations │
│ .py:757 in __getattr__                                                                           │
│                                                                                                  │
│    754 │   │   │   │   f"{type(self).__name__} object has no attribute {key!r}, did you mean {h  │
│    755 │   │   │   )                                                                             │
│    756 │   │                                                                                     │
│ ❱  757 │   │   raise AttributeError(f"'Table' object has no attribute {key!r}")                  │
│    758 │                                                                                         │
│    759 │   def __dir__(self) -> list[str]:                                                       │
│    760 │   │   out = set(dir(type(self)))                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Table' object has no attribute 'connection'

In [30]:
customers = con.sql("""
                         SELECT id as customer_id,
                         first_name,
                         last_name from raw_customers""").alias("customers")

In [31]:
ibis.to_sql(customers)

```sql
WITH "customers" AS (
  SELECT
    id AS customer_id,
    first_name,
    last_name
  FROM raw_customers
)
SELECT
  *
FROM "customers"
```

In [32]:
orders = con.sql("""SELECT id as order_id,
                 user_id as customer_id,
                 order_date,
                 status FROM raw_orders""").alias("orders")

In [33]:
payments = con.sql("""SELECT id as payment_id,
                   order_id,
                   payment_method,
                   amount / 100 as amount_in_dollars
                   from raw_payments""").alias("payments")

In [ ]:
['credit_card', 'coupon', 'bank_transfer', 'gift_card']

In [35]:
order_payments = payments.sql("""
    SELECT order_id,
    sum(case when payment_method = 'credit_card' then amount_in_dollars else 0 end) as credit_card_amount,
    sum(case when payment_method = 'coupon' then amount_in_dollars else 0 end) as coupon_amount,
    sum(case when payment_method = 'bank_transfer' then amount_in_dollars else 0 end) as bank_transfer_amount,
    sum(case when payment_method = 'gift_card' then amount_in_dollars else 0 end) as gift_card_amount,
    sum(amount_in_dollars) as total_amount
    from payments
    group by order_id

                              """)

In [23]:
ibis.to_sql(order_payments)

```sql
WITH "payments" AS (
  SELECT
    id AS payment_id,
    order_id,
    payment_method,
    amount / 100 AS amount_in_dollars
  FROM raw_payments
)
SELECT
  order_id,
  SUM(CASE WHEN payment_method = 'credit_card' THEN amount_in_dollars ELSE 0 END) AS credit_card_amount,
  SUM(CASE WHEN payment_method = 'coupon' THEN amount_in_dollars ELSE 0 END) AS coupon_amount,
  SUM(CASE WHEN payment_method = 'bank_transfer' THEN amount_in_dollars ELSE 0 END) AS bank_transfer_amount,
  SUM(CASE WHEN payment_method = 'gift_card' THEN amount_in_dollars ELSE 0 END) AS gift_card_amount,
  SUM(amount_in_dollars) AS total_amount
FROM payments
GROUP BY
  order_id
```

In [37]:
final = order_payments.sql("""
                select
        orders.order_id,
        orders.customer_id,
        orders.order_date,
        orders.status,
        order_payments.total_amount as amount

    from orders

    left join order_payments
        on orders.order_id = order_payments.order_id""").alias("final")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 final = order_payments.sql("""                                                              │
│    2 │   │   │   │   select                                                                      │
│    3 │   │   orders.order_id,                                                                    │
│    4 │   │   orders.customer_id,                                                                 │
│                                                                                                  │
│ /Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/ibis/expr/types/relations │
│ .py:3441 in sql                                                                                  │
│                                                                                                  │
│   3438 │   │   │   name = util.gen_name("sql_query")                                             │
│   3439 │   │   │   expr = self                                                                   │
│   3440 │   │                                                                                     │
│ ❱ 3441 │   │   schema = backend._get_sql_string_view_schema(name=name, table=expr, query=query)  │
│   3442 │   │   node = ops.SQLStringView(child=self.op(), query=query, schema=schema)             │
│   3443 │   │   return node.to_expr()                                                             │
│   3444                                                                                           │
│                                                                                                  │
│ /Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/ibis/backends/sql/__init_ │
│ _.py:198 in _get_sql_string_view_schema                                                          │
│                                                                                                  │
│   195 │   │   self, *, name: str, table: ir.Table, query: str                                    │
│   196 │   ) -> sch.Schema:                                                                       │
│   197 │   │   sql = self.compiler.add_query_to_expr(name=name, table=table, query=query)         │
│ ❱ 198 │   │   return self._get_schema_using_query(sql)                                           │
│   199 │                                                                                          │
│   200 │   def _register_udfs(self, expr: ir.Expr) -> None:                                       │
│   201 │   │   udf_sources = []                                                                   │
│                                                                                                  │
│ /Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/ibis/backends/duckdb/__in │
│ it__.py:1594 in _get_schema_using_query                                                          │
│                                                                                                  │
│   1591 │   │   │   pass                                                                          │
│   1592 │                                                                                         │
│   1593 │   def _get_schema_using_query(self, query: str) -> sch.Schema:                          │
│ ❱ 1594 │   │   with self._safe_raw_sql(f"DESCRIBE {query}") as cur:                              │
│   1595 │   │   │   rows = cur.fetch_arrow_table()                                                │
│   1596 │   │                                                                                     │
│   1597 │   │   rows = rows.to_pydict()                     